# Usuarios y títulos

In [1]:
import ast
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Préstamos

In [2]:
%%time

merge_df = pd.read_csv('merge.csv', header=0)
merge_df = merge_df.fillna('')

CPU times: user 1.5 s, sys: 71.3 ms, total: 1.57 s
Wall time: 1.57 s


In [3]:
merge_df.shape

(516206, 12)

In [4]:
merge_df.head()

,TITULO,AUTOR,DEWEY,FECHA_PUB,PAIS_EJEMPLAR,COD_BARRAS_LECTOR,CODBAR,Sucursal,Tratamiento,Edad,Ciudad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",925,2010,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",843,2009,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",986,2009,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49
3,Maravillas del mundo,"Steele, Philip.",910,2007,Mexico,88007424,88007424,ARM,sr.,43,Armenia,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",972,2008,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49


## Counts

#### Número de préstamos a socios activos

In [5]:
len(merge_df)

516206

#### Número de socios activos que tuvieron préstamos

In [6]:
merge_df.CODBAR.nunique()

29445

#### Número de préstamos y socios activos por ciudad

In [7]:
ciudades_df = pd.DataFrame({
        'Préstamos': merge_df.Ciudad.value_counts(),
        'Usuarios': merge_df.drop_duplicates('CODBAR').Ciudad.value_counts()
    })
ciudades_df['Préstamos / Usuarios'] = ciudades_df.apply(lambda x: x['Préstamos'] / x['Usuarios'], axis=1)
ciudades_df

,Préstamos,Usuarios,Préstamos / Usuarios
Armenia,8749,311,28.131833
Barranquilla,3745,222,16.869369
Bogotá,201558,13688,14.725161
Bucaramanga,10643,319,33.363636
Buenaventura,1849,122,15.155738
Cali,2266,62,36.548387
Cartagena,8741,565,15.470796
Cúcuta,648,18,36.000000
Florencia,9179,492,18.656504
Girardot,6980,685,10.189781


In [8]:
ciudades_df['Préstamos'].sum()

516206

In [9]:
ciudades_df['Usuarios'].sum()

29445

In [10]:
ciudades_df.sort_values('Préstamos', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
Bogotá,201558,13688,14.725161
Tunja,70583,2906,24.288713
Popayán,38816,1699,22.846380
Pasto,38011,1762,21.572645
Pereira,24129,1397,17.272011
Neiva,19008,1036,18.347490
Ibagué,14456,670,21.576119
Ipiales,13156,865,15.209249
Bucaramanga,10643,319,33.363636
Florencia,9179,492,18.656504


In [11]:
ciudades_df.sort_values('Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
Bogotá,201558,13688,14.725161
Tunja,70583,2906,24.288713
Pasto,38011,1762,21.572645
Popayán,38816,1699,22.846380
Pereira,24129,1397,17.272011
Neiva,19008,1036,18.347490
Ipiales,13156,865,15.209249
Girardot,6980,685,10.189781
Ibagué,14456,670,21.576119
Cartagena,8741,565,15.470796


In [12]:
ciudades_df.sort_values('Préstamos / Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
Villavicencio,660,13,50.769231
Manizales,3359,75,44.786667
Medellín,5384,143,37.650350
Cali,2266,62,36.548387
Cúcuta,648,18,36.000000
Bucaramanga,10643,319,33.363636
Santa Marta,4581,144,31.812500
Armenia,8749,311,28.131833
Tunja,70583,2906,24.288713
Popayán,38816,1699,22.846380


#### Número de préstamos y socios activos por rango de edad

In [13]:
edades_df = pd.DataFrame({
        'Préstamos': merge_df.Rango_edad.value_counts(),
        'Usuarios': merge_df.drop_duplicates('CODBAR').Rango_edad.value_counts()
    })
edades_df['Préstamos / Usuarios'] = edades_df.apply(lambda x: x['Préstamos'] / x['Usuarios'], axis=1)
edades_df

,Préstamos,Usuarios,Préstamos / Usuarios
+65,16798,825,20.361212
-17,140648,8128,17.304134
18-29,166938,11558,14.443502
30-39,96479,5090,18.954617
40-49,46757,2015,23.204467
50-64,48586,1829,26.564243


In [14]:
edades_df['Préstamos'].sum()

516206

In [15]:
edades_df['Usuarios'].sum()

29445

In [16]:
edades_df.sort_values('Préstamos', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
18-29,166938,11558,14.443502
-17,140648,8128,17.304134
30-39,96479,5090,18.954617
50-64,48586,1829,26.564243
40-49,46757,2015,23.204467
+65,16798,825,20.361212


In [17]:
edades_df.sort_values('Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
18-29,166938,11558,14.443502
-17,140648,8128,17.304134
30-39,96479,5090,18.954617
40-49,46757,2015,23.204467
50-64,48586,1829,26.564243
+65,16798,825,20.361212


In [18]:
edades_df.sort_values('Préstamos / Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
50-64,48586,1829,26.564243
40-49,46757,2015,23.204467
+65,16798,825,20.361212
30-39,96479,5090,18.954617
-17,140648,8128,17.304134
18-29,166938,11558,14.443502


#### Número de préstamos y socios activos por tratamiento

In [19]:
tratamientos_df = pd.DataFrame({
        'Préstamos': merge_df.Tratamiento.value_counts(),
        'Usuarios': merge_df.drop_duplicates('CODBAR').Tratamiento.value_counts()
    })
tratamientos_df['Préstamos / Usuarios'] = tratamientos_df.apply(lambda x: x['Préstamos'] / x['Usuarios'], axis=1)
tratamientos_df

,Préstamos,Usuarios,Préstamos / Usuarios
sr.,283048,14539,19.468189
sra.,233158,14906,15.641889


In [20]:
tratamientos_df['Préstamos'].sum()

516206

In [21]:
tratamientos_df['Usuarios'].sum()

29445

In [22]:
tratamientos_df.sort_values('Préstamos', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
sr.,283048,14539,19.468189
sra.,233158,14906,15.641889


In [23]:
tratamientos_df.sort_values('Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
sra.,233158,14906,15.641889
sr.,283048,14539,19.468189


In [24]:
tratamientos_df.sort_values('Préstamos / Usuarios', ascending=False)

,Préstamos,Usuarios,Préstamos / Usuarios
sr.,283048,14539,19.468189
sra.,233158,14906,15.641889
